<a href="https://colab.research.google.com/drive/1dUUN8StWSvQiVKF_4GzrmVuh6f1EeBCG#scrollTo=8AoNDCXvlgYe" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Минзадинов Ислам гр 6133

## Задание на Лабораторную работу №2


1. Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

2. Получившийся отчёт сохранить в формате Apache Parquet.

### 1. Настройка окружения

In [1]:
# устанавливаю pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=fa92815cbf53c80f0e1e3aaf329554a82d844619260db50b0e425058d5ecd35f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Добавляю гугл диск с данными
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyspark
import os
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

### 2. Создаю проект и проверяю его работо способность

In [4]:
# Создание точки конфигурации
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("L2ApacheReports").master("local[*]").getOrCreate()
sc

### 3. Загрузка данных

In [5]:
listProgLanguages = sc.read.csv("/content/drive/MyDrive/BigData/L2/programming-languages.csv")

Получим список языков

In [6]:
languageList = [str(x[0]) for x in listProgLanguages.collect()]
languageList[:5]

['name', 'A# .NET', 'A# (Axiom)', 'A-0 System', 'A+']

Удалим первый элемент `name`

In [7]:
languageList.pop(0)

'name'

In [8]:
languageList[:5]

['A# .NET', 'A# (Axiom)', 'A-0 System', 'A+', 'A++']

In [10]:
postsSampleData = sc.read.format("xml").options(rowTag="row").load('/content/drive/MyDrive/BigData/L2/posts_sample.xml')

In [11]:
postsSampleData.take(1)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

## Задание 1. Формирование отчетов.

Функция фильтрации ЯП

In [12]:
def detectionProgLanguage(x):
    tag = next((language for language in languageList if "<" + language.lower() + ">" in x._Tags.lower()), None)
    return None if tag is None else (x._Id, tag)

Реализуем функцию фильтрации по датам для получения списков топа по годам.

In [13]:
def detectionYear(x, year):
    start = datetime(year=year, month=1, day=1)
    end = datetime(year=year, month=12, day=31)
    return start <= x._CreationDate <= end

In [14]:
from pyspark.sql import functions as F

selection_prog_language = {}
years = range(2010, 2020)

for year in years:
    selection_prog_language[year] = (
        postsSampleData.rdd
        .filter(lambda x: x._Tags is not None and detectionYear(x, year))
        .map(detectionProgLanguage)
        .filter(lambda x: x is not None)
        .keyBy(lambda x: x[1])
        .aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2)
        .sortBy(lambda x: x[1], ascending=False)
        .toDF()
    )
    selection_prog_language[year] = (
        selection_prog_language[year]
        .select(F.col("_1").alias("Programming_language"), F.col("_2").alias(f"Number_of_mentions_in_{year}"))
        .limit(10)
    )

Выведем отчеты

In [18]:
for year in range(2010, 2020):
  selection_prog_language[year].show()

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2010|
+--------------------+--------------------------+
|                Java|                        52|
|          JavaScript|                        44|
|                 PHP|                        42|
|              Python|                        25|
|         Objective-C|                        22|
|                   C|                        20|
|                Ruby|                        11|
|              Delphi|                         7|
|                   R|                         3|
|                Bash|                         3|
+--------------------+--------------------------+

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2011|
+--------------------+--------------------------+
|                 PHP|                        97|
|                Java|                        92|
|          JavaScript|                        82|

## Задание 2. Сохранение отчетов.

In [17]:
for year in selection_prog_language.keys():
    save_path = f"/content/drive/MyDrive/BigData/L2/top_{year}"

    if os.path.exists(save_path):
        print(f"Файл {save_path} уже существует.")
    else:
        selection_prog_language[year].write.format("parquet").save(save_path)
        print(f"Сохранено в файл {save_path}.")

Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2010.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2011.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2012.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2013.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2014.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2015.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2016.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2017.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2018.
Сохранено в файл /content/drive/MyDrive/BigData/L2/top_2019.
